## Эксперименты
Здесь происходит идейно то же самое, но с вариациями в параметрах, описанных в статье. За все комбинации меня колаб забанит, поэтому приведу некоторые

Что от нас хотят в статье?


In [2]:
import torch
import nltk
from nltk import word_tokenize
#nltk.download('punkt')

# Скачиваем данные

In [4]:
import pandas as pd
data = pd.read_csv('Fake.csv')
data

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


# Предобученные эмбеддинги

In [6]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm import tqdm

In [7]:
# потом можете добавить свою предобработку

def process_text(text):
    
    words = wordpunct_tokenize(text.lower())
    
    return words

In [8]:
word2freq = {}
lengths = []

for text in tqdm(data.text):
    
    words = process_text(text)
    
    lengths.append(len(words))
    
    for word in words:
        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1

100%|█████████████████████████████████████████████████| 23481/23481 [00:06<00:00, 3656.98it/s]


# Читаем файл с эмбеддингами
### Этот файл с 300 числами для 2 000 000 слов и он может не влезть в память
Поэтому прочитаем только те слова, которые мы знаем

In [14]:
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [15]:
word2index = {'PAD': 0, 'UNK': 1}
vectors = []

n_words, embedding_dim = len(model.index_to_key), model.vector_size

#паддинги и уникальные
vectors.append(np.random.uniform(-0.25, 0.25, embedding_dim))
vectors.append(np.random.uniform(-0.25, 0.25, embedding_dim))
#progress_bar = tqdm(desc='Read word2vec', total=n_words)

for word in tqdm(word2freq.keys()):
    word2index[word] = len(word2index)
    if word in model:
        vectors.append(model.get_vector(word))
    else:
        vectors.append(np.random.uniform(-0.25, 0.25, embedding_dim))

vectors = torch.FloatTensor(np.array(vectors))

100%|████████████████████████████████████████████████| 95413/95413 [00:01<00:00, 51181.70it/s]


In [18]:
import torch

In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Подготовим данные в DataLoader

In [20]:
from torch.utils.data import Dataset, DataLoader

In [23]:
cat_mapper = {sub: n for n, sub in enumerate(data.subject.unique())}

In [24]:
cat_mapper

{'News': 0,
 'politics': 1,
 'Government News': 2,
 'left-news': 3,
 'US_News': 4,
 'Middle-east': 5}

In [25]:
data.subject = data.subject.map(cat_mapper)

# Читалка данных

In [26]:
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, word2index, sequence_length=512, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = y_data
        
        self.word2index = word2index
        self.sequence_length = sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(x_data, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
        
        words = wordpunct_tokenize(text.lower())
        #words = re.findall('[a-яА-ЯеЁ]+', text.lower())
        return words
        
    def load(self, data, verbose=True):
        
        data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
        for text in data_iterator:
            
            words = self.process_text(text)
            
            indexed_words = self.indexing(words)
            
            self.x_data.append(indexed_words)
    
    def indexing(self, tokenized_text):

        # здесь мы не используем токен UNK, потому что мы его специально не учили
        # становится непонятно какой же эмбеддинг присвоить неизвестному слову,
        # поэтому просто выбрасываем наши неизветсные слова
        
        return [self.word2index[word]  if word in self.word2index else self.word2index['UNK'] for word in tokenized_text]
    
    def padding(self, sequence):
        
        # Ограничить длину self.sequence_length
        # если длина меньше максимально - западить
        if len(sequence)< self.sequence_length:
          add_pad = self.sequence_length - len(sequence)
          return sequence+[self.pad_index]*add_pad
        else:
          return sequence[:self.sequence_length]
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        
        return x, y

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [28]:
x_train, x_validation, y_train, y_validation = train_test_split(data.text, data.subject, test_size=0.2)

train_dataset = WordData(list(x_train), list(y_train), word2index)
train_loader = DataLoader(train_dataset, batch_size=16)

validation_dataset = WordData(list(x_validation), list(y_validation), word2index)
validation_loader = DataLoader(validation_dataset, batch_size=16)

Loading data: 100%|█████████████████████████████████████| 4697/4697 [00:01<00:00, 2948.74it/s]


# Эксперименты

In [32]:
from math import sqrt

class model_clstm(torch.nn.Module):
    def __init__(self, matrix_w, n, conv_out=150, conv_window=[2, 3, 4],
                 dropout = 0.5,  output_dim=6, lstm_memory_dim=150,
                 dropout_locate='lstm', max_length=512): #n - количетсво категорий
        
        super().__init__()

        self.n = n
        self.embedding_dim = matrix_w.shape[1]
        #self.hidden_size = hidden_size
        self.conv_out = conv_out
        self.conv_window = conv_window
        self.max_cnn_res = max_length - max(conv_window) + 1
        self.lstm_memory_dim = lstm_memory_dim
        self.dropout = dropout
        self.dropout_locate = dropout_locate
        
        self.emb_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(matrix_w))
        conv_layers = []
        for c in conv_window:
            conv_layers.append(torch.nn.Conv1d(self.embedding_dim, self.conv_out, kernel_size=(c,), padding='valid'))
        self.convs = torch.nn.ModuleList(conv_layers)

        
        self.LSTM = torch.nn.LSTM(input_size=self.conv_out * len(self.conv_window), 
                                  hidden_size=self.lstm_memory_dim,
                                  num_layers=1, batch_first=True)
        
        self.soft = torch.nn.Softmax(dim=1)
        self.relu = torch.nn.ReLU()
        self.linear = torch.nn.Linear(in_features=self.lstm_memory_dim,
                                out_features=self.n)
        self.dropout = torch.nn.Dropout(p=0.5)

        
    def forward(self, x):
        
        if self.dropout_locate == 'cnn':
            x_emb = self.emb_layer(x)
        else:
            x_emb = self.dropout(self.emb_layer(x))
            
        x_emb = x_emb.transpose(1,2)
        
        cnn_results = []
        for conv in self.convs:
            cnn_res = self.relu(conv(x_emb))[:, :, :self.max_cnn_res]
            cnn_results.append(cnn_res)
            
        if len(self.conv_window) > 1:
            to_lstm = torch.cat(cnn_results, 1)
        else:
            to_lstm = cnn_res
            
        to_lstm = to_lstm.transpose(1, 2)
        _, (x, _)= self.LSTM(to_lstm)
        x.squeeze_(0)
        
        if self.dropout_locate != 'cnn':           
            x = self.dropout(x)
        logits = self.soft(self.linear(x))
        return logits

In [33]:
def train_val(model_clstm, train_loader=train_loader, validation_loader=validation_loader):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(params=model_clstm.parameters(), lr=0.001, weight_decay=0.001)

    model_clstm = model_clstm.to(device)
    criterion = criterion.to(device)

    epochs = 5
    losses = []
    best_test_loss = 10

    test_accuracy = []

    for n_epoch in range(epochs):

        train_losses = []
        test_losses = []
        test_targets = []
        test_pred_class = []

        progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))

        model_clstm.train()

        for x, y in train_loader:

            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()

            pred = model_clstm(x)
            loss = criterion(pred, y)

            loss.backward()

            optimizer.step()

            train_losses.append(loss.item())
            losses.append(loss.item())

            progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

            progress_bar.update(x.shape[0])

        progress_bar.close()

        model_clstm.eval()

        for x, y in validation_loader:

            x = x.to(device)

            with torch.no_grad():

                pred = model_clstm(x)

                pred = pred.cpu()

                test_targets.append(y.numpy())
                test_pred_class.append(np.argmax(pred, axis=1))

                loss = criterion(pred, y)

                test_losses.append(loss.item())

        mean_test_loss = np.mean(test_losses)

        test_targets = np.concatenate(test_targets).squeeze()
        test_pred_class = np.concatenate(test_pred_class).squeeze()

        accuracy = accuracy_score(test_targets, test_pred_class)

        test_accuracy.append(accuracy)

        print()
        print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

        print('Accuracy test - {:.3f}'.format(accuracy))

        # Early stopping:
        #if mean_test_loss < best_test_loss:
        #    best_test_loss = mean_test_loss
        #else:
        #    print('Early stopping')
        #    break

## Эксперимент номер один

In [34]:
model_clstm = model_clstm(vectors, 6, conv_window=[3, 4], dropout_locate='cnn')

In [35]:
model_clstm

model_clstm(
  (emb_layer): Embedding(95415, 300)
  (convs): ModuleList(
    (0): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=valid)
    (1): Conv1d(300, 150, kernel_size=(4,), stride=(1,), padding=valid)
  )
  (LSTM): LSTM(300, 150, batch_first=True)
  (soft): Softmax(dim=1)
  (relu): ReLU()
  (linear): Linear(in_features=150, out_features=6, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [36]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [37]:
train_val(model_clstm)

Epoch 1: 100%|████████████████████████| 18784/18784 [01:34<00:00, 198.64it/s, train_loss=1.56]



Losses: train - 1.591, test - 1.550
Accuracy test - 0.493


Epoch 2: 100%|████████████████████████| 18784/18784 [01:26<00:00, 216.64it/s, train_loss=1.54]



Losses: train - 1.543, test - 1.519
Accuracy test - 0.527


Epoch 3: 100%|████████████████████████| 18784/18784 [01:30<00:00, 206.58it/s, train_loss=1.46]



Losses: train - 1.507, test - 1.439
Accuracy test - 0.604


Epoch 4: 100%|████████████████████████| 18784/18784 [01:28<00:00, 211.66it/s, train_loss=1.45]



Losses: train - 1.451, test - 1.433
Accuracy test - 0.609


Epoch 5: 100%|████████████████████████| 18784/18784 [01:28<00:00, 211.16it/s, train_loss=1.45]



Losses: train - 1.449, test - 1.445
Accuracy test - 0.597


Уже лучше, хотя начинает переобучаться -- 0.609

Если вы запускаете много раз колаб окна и ткдм начинает беситься, можно запустить окно ниже, ткдм обновится и все снова станет хорошо

In [38]:
for instance in list(tqdm._instances): 
    tqdm._decr_instances(instance)